# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [2]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('clientes2_merge.csv')

In [3]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [4]:
port1.shape

(566, 182)

In [5]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [6]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(566, 37)

In [ ]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [7]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [8]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [9]:
#market_p1.head(2)

## Encoding

In [10]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [11]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [12]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [13]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [14]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d,1,11.243836,0,0,0,0,0,1,1,0,...,0,0,0,4,0,0,17,57,6,6
dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e,1,43.789041,0,0,0,0,0,0,0,0,...,0,0,19,4,0,0,0,1,4,4
16843c9ffb920170477118933798510d8d5f00f5c56c1221773356355bded321,1,16.389041,0,0,0,0,0,0,1,0,...,36,11,16,4,0,0,9,11,3,3


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [59]:
market_sample = market_encoded#.sample(n=50000)

In [60]:
%%time
from sklearn.neighbors import NearestNeighbors

qtd_neighbors = 5
nearest = NearestNeighbors(n_neighbors=qtd_neighbors, metric = 'cosine')

nearest.fit(market_sample)


CPU times: user 52.9 ms, sys: 110 ms, total: 163 ms
Wall time: 186 ms


Dict para a lista de vizinhos mais próximos

In [61]:
%%time
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [62]:
neighbors_list

{0: (array([[0.00000000e+00, 1.63313807e-13, 3.18411963e-13, 8.56315019e-13,
          1.52167168e-12]]),
  array([[  1646,   8626,  95246,  51124, 439410]])),
 1: (array([[0.00000000e+00, 2.50133247e-13, 9.54458734e-13, 1.33559830e-12,
          1.38555833e-12]]),
  array([[  2696, 325674,   4564,  22688, 301752]])),
 2: (array([[3.33066907e-16, 1.04161124e-12, 2.79498646e-12, 4.87887508e-12,
          6.24167384e-12]]),
  array([[  3681,   6066, 424650,  29452, 105545]])),
 3: (array([[3.33066907e-16, 1.57873714e-13, 3.94684285e-13, 4.20885549e-13,
          6.11510842e-13]]),
  array([[  4564, 301752, 439410, 325674, 402582]])),
 4: (array([[2.22044605e-16, 1.04161124e-12, 1.62092562e-12, 2.19280150e-12,
          5.54656321e-12]]),
  array([[  6066,   3681, 424650,  29452, 105545]])),
 5: (array([[0.00000000e+00, 3.08387760e-11, 4.37001546e-11, 4.58292293e-11,
          4.77013984e-11]]),
  array([[  6345,  26577, 253044, 356504, 189784]])),
 6: (array([[0.00000000e+00, 3.56176200e

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [63]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
np.delete(neighbors_idx_array, [0,1])
np.delete(neighbors_distance_array, [0,1])
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

np.delete(neighbors_idx_array, [0,1])
np.delete(neighbors_distance_array, [0,1])

array([3.18411963e-13, 8.56315019e-13, 1.52167168e-12, ...,
       1.07007007e-07, 1.27437902e-07, 1.32935897e-07])

In [37]:
neighbors_distance_array[0]

1.5233356798205477e-09

Criar um dicionário com o número da linha, id e distância

In [64]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (port1.iloc[int(ind/qtd_neighbors)].name, market_sample.iloc[idx].name, (neighbors_distance_array[ind]))

In [66]:
dicio

{0: ('09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  '09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  0.0),
 1: ('09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  '194e619cb6417f92f75653406cf04106debc5035f13a8850b00428fd34d91d72',
  1.6331380692236053e-13),
 2: ('09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  'cf5abf9d1a1b5912e1fe2825f6130fdaa50a313a0d68f3693f675ad1d7e60bfc',
  3.184119634624949e-13),
 3: ('09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  'a74b4b5172bafa295c4d2ef1548da7ccfe3d5e3502d632a077b730f415793905',
  8.563150188933832e-13),
 4: ('09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d',
  '980ad70b2ba1aed92c6106505f630fc7477e587e9ebac378d073bbb92d305e94',
  1.5216716775512396e-12),
 5: ('dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e',
  'dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e',
  0.0),
 6: ('dc9d155f5bcd31

In [65]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'id_origin', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)

In [67]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [68]:
len(conflicts.index.unique()), conflicts.shape, suggestion_with_conflicts.shape, port1.shape, suggestions.shape

(566, (1202, 39), (2830, 39), (566, 37), (1628, 39))

In [69]:
suggestions.head(5)

,id_origin,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
0012f06c4ea31b52d25c6d791ec7f6c087bd7c0eb1f338c2400b987ee6e2c22b,d8e21466fc2ab4b21c3456be1594c93abb67fc49127a8d...,5.202584e-09,1,MUNICIPIO,RO,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,31.150685,> 20,...,other,other,1,0.0,0.0,0.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",104474450.0,105614450.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,6940a00c907814c5252c927e5f9d6e6f4ba29612bcd90a...,8.879453e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.0,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,5b0e32f5b3c58c14ccbba2da37b1b718ba25bee3bf1e30...,5.541567e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.0,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
0086ad83094c5903c631bb3028c022ec7f15391f7bbb7ba2d5036588db7547a1,7fc36495804caf524926b223b90e2e90eb713db15d1059...,2.450540e-11,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,SERVICOS ADMINISTRATIVOS,SERVIÇO,12.032877,10 a 15,...,CENTRO AMAZONENSE,MANAUS,1,1.0,67.0,0.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",12363840.0,12363840.0
00ac2b0f09807d4304312374622e5e6a99337ba14858d30f9f546dc01fc8c5de,5c7e219e68b5521c3268c287ee25e945d36e8fbfd0f2b6...,2.052691e-11,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,22.353425,> 20,...,LESTE MARANHENSE,CODO,0,3.0,66.0,1.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",16691184.0,16691184.0


# Visualização dos resultados

## Mapa (Heat)

### Carrega arquivo com informações geográficas
Gerado por `analysis/Save_IBGE_data_for_Map_Visualization_in_Recommendation_System.ipynb`

In [70]:
coord_micro = pd.read_csv('../recommender_system_squad_1/coord_geograficas_microrregioes_brasil.csv')

In [71]:
### Importa módulo Folium
import folium
from folium import plugins
import random

### Gera lista com coordenadas das sugestões (suggestions)

In [72]:
map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in suggestions['nm_micro_regiao']:
    coord = coord_micro[coord_micro['nm_micro'] == local][['lat', 'lng']]
    if not coord.empty:
        map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.25),
                          coord['lng'].values[0]-random.uniform(0, 0.25)])
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning
print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 225


### Mostra heat map

In [73]:
mapa = folium.Map(location=[-8.788497,-53.879873],tiles='Stamen Toner',zoom_start=5)

mapa.add_child(plugins.HeatMap(map_coord))
mapa

## Mapa com marcadores
Passe o mouse sobre os marcadores para informação adicional.

In [74]:
# Para não ter repetições no mapa com marcadores, vou criar um novo df a partir de suggestions,
# deixando só uma entrada para cada id do market
unique_suggestions = pd.DataFrame()
for client in suggestions['id_origin'].unique():
    idxmin_distance = suggestions[suggestions['id_origin']==client]['distance'].idxmin()
    unique_suggestions = unique_suggestions.append(market.loc[idxmin_distance])

In [92]:
uniq_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(unique_suggestions[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 unique_suggestions[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        uniq_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+local[1][:10]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 84


In [90]:
#incluir também clientes atuais
current_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(port1[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 port1[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        current_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+str(local[1])[:10]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Current clients without info about nm_micro:', null_count)

Current clients without info about nm_micro: 70


In [94]:
# Mostra marcadores com informação do id e ramo da empresa de sugestão.
markmapa = folium.Map(location=[-8.788497, -53.879873],
                  tiles='OpenStreetMap', zoom_start=5)

# tooltip = 'Click for info'
#clientes atuais
# for mark in current_map_coord:
#     folium.Marker([mark[0], mark[1]],
#                  popup = mark[2],
#                  tooltip = mark[2],
#                  icon=folium.Icon(icon='ok', color='green')).add_to(markmapa)
    
for mark in uniq_map_coord:
    # sugestões
    folium.Marker([mark[0], mark[1]], popup=mark[2], tooltip=mark[2]).add_to(markmapa)

markmapa.add_child(plugins.HeatMap(map_coord))
markmapa


## Lista ordenada por faixa de faturamento